In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

from bs4 import BeautifulSoup as bs
import requests

import pd_helpers as pdh

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pd_helpers'

In [ ]:
import pandas as 
def pull_bea():
    ### Make u_employment.csv
    geofips = 'COUNTY'
    linecode = '300' # Private nonfarm earnings: Utilities (22)
    # linecode = '905' # Private nonfarm earnings: Telecommunications (517) # not county-level
    # linecode = '800' # Private nonfarm earnings: Transportation and warehousing (48-49)
    # linecode = '811' # Private nonfarm earnings: Warehousing and storage (493) # not county-level
    tablename = 'CAEMP25N' # Total Full-Time and Part-Time Employment by NAICS Industry
    call = f'https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getdata&datasetname=Regional&geofips=%s&linecode=%s&tablename=%s&Year=ALL&ResultFormat=XML' % (geofips, linecode, tablename)

    call_xml = requests.get(call).text
    call_soup = bs(call_xml, 'lxml')
    call_list = [[line['geoname'], line['timeperiod'], line['datavalue']] for line in call_soup.find_all('data')]
    call_df = pd.DataFrame(data=call_list, columns=['County', 'Year', 'Number of Employees'])
    call_df = call_df.astype({'County': str, 'Year': int, 'Number of Employees': int})
    udf = call_df.groupby('County')['Number of Employees'].last() # use last year
    udf.to_csv('u_employment.csv')

    ### Make tw_compensation.csv
    geofips = 'COUNTY'
    # linecode = '300' # Private nonfarm earnings: Utilities (22)
    # linecode = '905' # Private nonfarm earnings: Telecommunications (517)
    linecode = '800' # Private nonfarm earnings: Transportation and warehousing (48-49)
    # linecode = '811' # Private nonfarm earnings: Warehousing and storage (493)
    # tablename = 'CAINC5N' # Personal Income by Major Component and Earnings by NAICS Industry
    tablename = 'CAINC6N' # Compensation of Employees by NAICS Industry 
    call = f'https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getdata&datasetname=Regional&geofips=%s&linecode=%s&tablename=%s&Year=ALL&ResultFormat=XML' % (geofips, linecode, tablename)

    call_xml = requests.get(call).text
    call_soup = bs(call_xml, 'lxml')
    call_list = [[line['geoname'], line['timeperiod'], line['datavalue']] for line in call_soup.find_all('data')]
    call_df = pd.DataFrame(data=call_list, columns=['County', 'Year', 'Dollars (Thousands)'])
    call_df = call_df.astype({'County': str, 'Year': int, 'Dollars (Thousands)': float})

    geofips = 'COUNTY'
    linecode = '800'
    tablename = 'CAEMP25N' # Total Full-Time and Part-Time Employment by NAICS Industry
    call = f'https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getdata&datasetname=Regional&geofips=%s&linecode=%s&tablename=%s&Year=ALL&ResultFormat=XML' % (geofips, linecode, tablename)

    call_xml = requests.get(call).text
    call_soup1 = bs(call_xml, 'lxml')
    call_list1 = [[line['geoname'], line['timeperiod'], line['datavalue']] for line in call_soup1.find_all('data')]
    call_df1 = pd.DataFrame(data=call_list1, columns=['County', 'Year', 'Number of Employees'])
    call_df1 = call_df1.astype({'County': str, 'Year': int, 'Number of Employees': float})

    df = pd.merge(left=call_df, right=call_df1, on=['County', 'Year'], how='inner')
    last_yr = df['Year'].max()
    df = df[df['Year'] >= last_yr - 5] # last 5 years
    df['Average Compensation'] = df['Dollars (Thousands)'] * 1000 / df['Number of Employees']
    df.replace(0, np.nan, inplace=True)
    df.dropna(inplace=True)
    comp = df.groupby('County').mean()['Average Compensation'] # use average of last 5 years
    employ = df.groupby('County').last()['Number of Employees'] # use last non-zero
    df1 = pd.DataFrame([comp, employ]).transpose()
    df1.rename(columns={'Number of Employees': f'Number of Employees %i' % last_yr}, inplace=True)
    df1.to_csv('tw_compensation.csv')

def make_tw_comp_by_zip(tw_comp_path='tw_compensation.csv', zip_info_path='zip_census.csv', min_comp=25000, max_comp=65000):
    bea = pd.read_csv(tw_comp_path, index_col=0)

    zip_info = pd.read_csv(zip_info_path, dtype={'zip': str})
    zip_info = zip_info[~zip_info['military']]
    zip_info = zip_info[~zip_info['state_id'].isin(['AK', 'HI', 'DC','GU','PR','VI','MP','AS'])]
    zip_info = zip_info[['zip', 'lat', 'lng', 'county_name', 'state_id', 'county_names_all']]
    zip_info['county_names_all'] = zip_info['county_names_all'].str.split('|')
    zip_info['county'] = zip_info['county_name'] + ', ' + zip_info['state_id']

    df = zip_info.copy()
    df['cntys_found'] = np.isin(df['county'], bea.index)
    # for ZIPs with an unfound county, try their second, third, and fourth listed counties if listed
    for i in range(1,4):
        county = []
        for ind, row in df.iterrows():
            if not row['cntys_found'] and len(row['county_names_all']) > i:
                county.append(row['county_names_all'][i] + ', ' + row['state_id'])
            else:
                county.append(row['county'])
        df['county'] = county
        df['cntys_found'] = np.isin(df['county'], bea.index)

    # for ZIPs with an unfound county or county with zero value, replace with the state's average
    df['tw_compensation'] = [bea['Average Compensation'].loc[df['county'].iloc[i]] if df['cntys_found'].iloc[i] else np.nan for i in range(len(df))]
    tw_sal = []
    for ind, row in df.iterrows():
        if not row['cntys_found'] or row['tw_compensation'] == 0:
            tw_sal.append(df[df['state_id'] == row['state_id']]['tw_compensation'].mean())
        else:
            tw_sal.append(row['tw_compensation'])
    df['tw_compensation'] = tw_sal

    # min and max compensation
    df['tw_compensation'] = np.where(df['tw_compensation'] < min_comp, min_comp, df['tw_compensation'])
    df['tw_compensation'] = np.where(df['tw_compensation'] > max_comp, max_comp, df['tw_compensation'])

    df[['zip', 'tw_compensation']].to_csv('tw_compensation_by_zip.csv', index=False)

def make_invoice_item(cust_path="GT - Customer Dataset.csv", item_path="GT - Item Dataset.csv", sales_path="GT - Sales (Invoiced) Dataset.csv", 
                cust_del="cust_del.xlsx", cust_use_bill_addr="cust_use_bill_addr.xlsx"):
    make_invoice_dol(cust_path, item_path, sales_path, cust_del, cust_use_bill_addr, True)
    
def make_invoice_dol(cust_path="GT - Customer Dataset.csv", item_path="GT - Item Dataset.csv", sales_path="GT - Sales (Invoiced) Dataset.csv", 
                cust_del="cust_del.xlsx", cust_use_bill_addr="cust_use_bill_addr.xlsx", output_item=False):
    customer = pd.read_csv(cust_path, header=0, index_col="Entity ID").drop_duplicates()
    customer.drop(columns=['Address 3'], inplace=True)
    item = pd.read_csv(item_path, header=0).drop_duplicates()
    sales = pd.read_csv(sales_path, header=0).drop_duplicates()

    if cust_del:
        custs_d = pd.read_excel(cust_del).astype('str').values.flatten()
        customer.drop(index=custs_d, errors='ignore', inplace=True)

    if cust_use_bill_addr:
        custs_b = pd.read_excel(cust_use_bill_addr).astype('str').values.flatten()
        custs_b_df = customer.loc[custs_b]
        custs_b_df["Address"] = custs_b_df["Default Billing Address"]
        cust = customer.drop(index=custs_b, errors='ignore')
        cust["Address"] = cust["Default Shipping Address"]
        customer = pd.concat([custs_b_df, cust]).drop(columns=["Default Shipping Address", "Default Billing Address"])
        customer["Address"] = customer["Address"].astype(str)
    else:
        customer["Address"] = customer["Default Shipping Address"]

    customer["Address"] = customer["Address"].str.removeprefix("Validate Address\n")

    addr = []
    for i in range(len(customer)):
        splt = customer["Address"].iloc[i].split("\n")
        addr_i = list(np.empty(6, dtype=str))
        for j in range(len(splt)):
            addr_i[5-j] = splt[len(splt)-1-j]
        addr.append(addr_i)

    addr = np.array(addr)
    # print(addr[addr[:,5] != "United States"])

    customer["Zip Description"] = addr[:,4]
    customer["Address"] = customer["Address"].str.replace("\n", " ")

    # make customer the ID only
    sales["Customer"] = sales["Customer"].apply(lambda x: str.split(x)[0])
    # make amount a float
    sales["Amount"] = sales["Amount"].str.replace("$", "").str.replace(",", "").astype(float)

    merged0 = pd.merge(sales, customer, how="left", left_on="Customer", right_on="Entity ID")

    merged = pd.merge(merged0, item, how="left", left_on="Item", right_on="Item Name/Number")
    merged.drop(columns=["Country", "Item Name/Number", "Carrier", "Parent"], inplace=True)
    merged.rename(columns={"Customer": "Customer ID", "Company Name": "Customer Name", "Item Weight ": "Item Weight"}, inplace=True)    

    if output_item:
        # drop blank main lines
        merged = merged[~(merged["Item"].isna() & merged["Amount"] == 0)]

        merged.to_csv("invoice_items.csv", index=False)
    else:
        merged.fillna(0, inplace=True)
        gby = merged.groupby(by=["Date", "Customer ID", "Location", "Customer Name", "City", "State", "Zip", "Address", "Zip Description"]).agg({'Amount': 'sum'})
        gby.reset_index(inplace=True)
        gby['Amount'] = gby['Amount'].round(2)
        gby.to_csv("invoice_dol.csv", index=False)

def make_geo(lvl='zip2', zip_info_path='zip_census.csv', u_employ_path='u_employment.csv', out_path=''):
    if lvl not in ['zip3', 'zip2']:
        return
    
    zip_info = pd.read_csv(zip_info_path, dtype={'zip': str})
    zip_info = zip_info[~zip_info['military']]
    zip_info = zip_info[~zip_info['state_id'].isin(['AK', 'HI', 'DC','GU','PR','VI','MP','AS'])]

    df = zip_info.copy()
    df['zip2'] = df['zip'].str.slice(stop=2)
    df['zip3'] = df['zip'].str.slice(stop=3)
    df['county_weights'] = df['county_weights'].apply(lambda dict_str: list(eval(dict_str).values()))

    ### Make a DataFrame of zip2/zip3s, their populations, and counties
    data = dict()
    z2_pop = []
    cnty_weight = []
    cnty_lists = []
    # multi_st = 0 # count number of ZIPs with multiple states
    for zip2 in df[lvl].unique():
        z2_df = df[df[lvl] == zip2]
        # if len(z2_df['state_id'].unique()) > 1:
        #     multi_st += 1
        counties = z2_df['county_names_all'].str.split('|')
        cnty_list = sum(counties.values, [])
        cnty_list = list(set(cnty_list))
        cnty_pop = {c: 0 for c in cnty_list}
        for ind in z2_df.index:
            for i in range(len(counties.loc[ind])):
                cnty_pop[counties.loc[ind][i]] += z2_df['county_weights'].loc[ind][i]/100 * z2_df['population'].loc[ind]
        pop = z2_df['population'].sum()
        z2_pop.append(pop)
        cnty_st = {c: z2_df['state_id'].iloc[0] if not len(z2_df[z2_df['county_name'] == c]['state_id'])
                else z2_df[z2_df['county_name'] == c]['state_id'].iloc[0] for c in cnty_list}
        cnty_weight.append({c + ', ' + cnty_st[c]: cnty_pop[c] / pop for c in cnty_list})
        cnty_lists.append([c + ', ' + cnty_st[c] for c in cnty_list])
        cnty_pop = {c + ', ' + cnty_st[c]: cnty_pop[c] for c in cnty_pop}
        data[zip2] = cnty_pop
    z2 = pd.DataFrame({lvl: data.keys(), 'pop': z2_pop, 'counties': cnty_lists, 'county_pops': data.values(), 'county_weights': cnty_weight})

    ### Make DataFrame of counties and total populations according to zip_info
    county1 = np.concatenate([list(z2['county_pops'].iloc[i].keys()) for i in range(len(z2))])
    pop1 =  np.concatenate([list(z2['county_pops'].iloc[i].values()) for i in range(len(z2))])
    county_pop = pd.DataFrame({'county': county1, 'pop': pop1}).groupby(by='county').sum()

    ### Add the percent of each county that is in the zip2/zip3
    data = []
    for i in range(z2.shape[0]):
        dct = z2['county_pops'].iloc[i]
        data.append({c: (dct[c]/county_pop.loc[c])[0] for c in dct})
    z2['county_percents'] = data

    ### Use the percent of each county inside the zip2/zip3 to aggregate the county-level
    ###     u_employment values into one value for each zip2/zip3
    bea = pd.read_csv(u_employ_path, index_col=0)

    col = 'Number of Employees'
    # bad_counties = set()
    # zero_counties = set()
    new_col = []
    for i in range(len(z2)):
        cntys = np.array(list(z2['county_percents'].iloc[i].keys()))
        wghts = np.array(list(z2['county_percents'].iloc[i].values()))
        cntys_found = np.isin(cntys, bea.index)
        # print(wghts.shape[0], cntys_found.shape[0])
        # bad_counties.update(cntys[~cntys_found])
        cntys1 = cntys[cntys_found]
        wghts1 = wghts[cntys_found] / wghts[cntys_found].sum()
        # print(wghts1.shape[0], cntys1.shape[0])
        # zero_counties.update(cntys1[cntys1==0])
        u = np.array(bea[col].loc[cntys1].values).flatten()
        u1 = np.where(u==0, 0.1, u)
        new_col.append(np.dot(wghts1, u1))
    z2['u_employ'] = new_col
    # print(len(bad_counties), len(zero_counties), len(bea2))
    
    lats = []
    lngs = []
    for zip2 in df[lvl].unique():
        z2_df = df[df[lvl] == zip2]
        lats.append(np.dot(z2_df['population'], z2_df['lat'])/z2_df['population'].sum())
        lngs.append(np.dot(z2_df['population'], z2_df['lng'])/z2_df['population'].sum())
    z2['lat'] = lats
    z2['lng'] = lngs

    if not out_path:
        out_path = lvl + '_geo.csv'
    if out_path[-5] == '.xlsx':
        z2.to_excel(out_path, index=False)
    z2.to_csv(out_path, index=False)

def make_geo_sales(lvl='zip2', geo_path='', sales_path='invoice_dol.csv', w_path='existing_locations.xlsx', features=['u_employ']):
    if lvl not in ['zip3', 'zip2']:
        return
    if not geo_path:
        geo_path = lvl + '_geo.csv'

    if geo_path[-5:] == '.xlsx':
        geo = pd.read_excel(geo_path, dtype={lvl: str})[[lvl, *features, 'lat','lng']]
    else: # geo_path[-4:] == '.csv'
        geo = pd.read_csv(geo_path, dtype={lvl: str})[[lvl, *features, 'lat','lng']]
    
    geo.dropna(subset=['lat', 'lng'], inplace=True)

    if sales_path[-5:] == '.xlsx':
        sales = pd.read_csv(sales_path, dtype={'Zip': str}, parse_dates=['Date'])
    else: # sales_path[-4:] == '.csv'
        sales = pd.read_csv(sales_path, dtype={'Zip': str}, parse_dates=['Date'])

    # w_path[-4:] == '.csv'
    ws = pd.read_excel(w_path, dtype={'zip': str}) # one row for each (warehouse, sales center) pair
    w = ws.groupby('name').sum().reset_index().drop(columns=['sales_name']) # one row for each warehouse
    
    # aggregate dates to month-level
    sales = pdh.daily_to_monthly(sales, date_col="Date", cat_col=['Customer ID', 'State', 'Zip', 'Location'])
    # filter to last 12 months of full data
    sales = pdh.rolling_1yr(sales)

    # change Location from column of Fulfillment Centers and Sales Centers to only Fulfillment Centers
    warehouse_map = {ws['sales_name'].iloc[i]: ws['name'].iloc[i] for i in range(ws.shape[0])}
    sales['warehouse'] = [warehouse_map[loc] if loc in warehouse_map.keys() else loc for loc in sales['Location']]

    sales = sales[['date', 'warehouse', 'Customer ID', 'Zip', 'State', 'Amount']]
    sales.rename(columns={'State': 'st'})
    sales['zip3'] = sales['Zip'].str.slice(stop=3)
    sales['zip2'] = sales['Zip'].str.slice(stop=2)

    # group sales by geo lvl, warehouse, and date
    df = sales.groupby([lvl, 'warehouse', 'date']).sum(numeric_only=True).reset_index()

    # only include warehouses in w (drop FL)
    df = df[np.isin(df['warehouse'], w['name'])]

    # add rows so that all values of lvl in geo are in df, adding 0s for missing values
    df = pdh.full_col(df, lvl, geo[lvl].unique())

    # add rows so that all combinations of [lvl, 'warehouse', 'date] are in df
    df = pdh.full_df(df, cat_col=[lvl, 'warehouse','date'])
    df = df[(df['warehouse'] != "0") & (df['date'] != pd.to_datetime(0))]
    df = df.sort_values(by=[lvl, 'warehouse', 'date']).reset_index(drop=True)

    # merge geo, sales, w
    w_coord = w[['name', 'lat', 'lng']]
    df = pd.merge(left=df, right=w_coord, left_on='warehouse', right_on='name', how='left')
    lvl_df = pd.merge(left=df, right=geo, on=lvl, suffixes=('_w', '_g'))
    lvl_df1 = lvl_df[[lvl, 'warehouse', 'lat_w', 'lng_w', 'lat_g', 'lng_g']].drop_duplicates()

    # calculate distances on the smaller version lvl_df1
    lvl_df1['dist'] = [geodesic((lvl_df1['lat_g'].iloc[i], lvl_df1['lng_g'].iloc[i]), (lvl_df1['lat_w'].iloc[i], lvl_df1['lng_w'].iloc[i])).miles for i in range(lvl_df1.shape[0])]

    # merge the distances back into the full lvl_df so the distances are repeated
    lvl_df2 = pd.merge(left=lvl_df, right=lvl_df1[[lvl, 'warehouse', 'dist']], on=[lvl, 'warehouse']).drop(columns='name')

    # calculate ms = $ sold / # utilities employees
    lvl_df2['Amount'] = lvl_df2['Amount'].astype('float64').round(2)
    lvl_df2['ms'] = lvl_df2['Amount'] / lvl_df2['u_employ']

    lvl_df2.to_csv('%s_geo_sales.csv' % (lvl), index=False)

### run functions
# pull_bea()
# make_geo()
# make_tw_comp_by_zip()
# # make_invoice_item()
# make_invoice_dol()
# make_geo_sales()
# make_geo('zip3')
# make_geo_sales('zip3')